In [4]:
from pipeline import ClaimifyPipeline
from llm_client import LLMClient
import polars as pl
import json
import os

classification_data_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data_for_git/covid_classifications.jsonl")

# load with utf-8 encoding
articles = []
with open(classification_data_path, "r", encoding="utf-8") as f:
    for line in f:
        articles.append(json.loads(line))

article_data_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data/article_texts_with_metadata.parquet")

article_data = pl.read_parquet(article_data_path)

print(articles[0]["result"])

kategorier = set([article["result"]["huvudkategori"] for article in articles])

kategori = "Svenska strategin"
selected_articles = []
for article in articles:
    if article["result"]["huvudkategori"] == kategori:
        selected_articles.append(article)

print(len(selected_articles))

combined_data = []
for article in selected_articles:
    combined = {"classification": article["result"], "article": article_data.filter(pl.col("id") == article["id"]).to_dicts()[0]}
    combined_data.append(combined)

print(combined_data[1]["article"]["publication"])
print(combined_data[1]["article"]["body_text"])

{'resonemang': 'Artikeln rapporterar om andelen personer med antikroppar mot covid-19 i olika svenska städer, vilket handlar om befolkningens immunitet mot viruset.', 'kategorier': ['Immunitet'], 'huvudkategori': 'Immunitet', 'källor': None, 'personer': ['Henrik Forsberg'], 'länder': ['Sverige']}
322
aftonbladet
Nya boken ”Flocken” avslöjar spelet bakom den så kallade svenska strategin.

Genom att gå igenom en mängd mejl har författaren Johan Anderberg pusslat ihop födelsen av statsepidemiolog Anders Tegnells covid-19-strategi. Svenska Dagbladet har fått tag på ett bearbetat, ännu opublicerat utdrag ur boken som beskriver i detalj spelet bakom det som skulle bli Sveriges förmodligen mest debatterade vägval året 2020.
Den 6 mars fick Anders Tegnell ett varnande mejl från matematikern Tom Britton som räknat på R-värdet för viruset. Där framförde han sin åsikt att  Folkhälsomyndigheten var för passiv: ”Med så många smittade som nu finns i Sverige så kommer smittspridning att ske. Vårt sam

In [5]:
import threading
os.environ["OPENAI_API_KEY"] = ""
model = "openai/gpt-oss-120b"
llm_client = LLMClient(model)

pipeline = ClaimifyPipeline(llm_client, question="Vad uttrycker texten om den svenska strategin för att hantera Coronapandemin?")
results = {}
result_lock = threading.Lock()


def run_pipeline(sample, result, result_lock):
    article = sample["article"]
    prompt = article["title"] + "\n" + article["actual_lead_text"] + "\n" + article["body_text"]
    result = pipeline.run(prompt)
    with result_lock:
        results[article["id"]] = result


threads = []
for i, sample in enumerate(combined_data[1:2]):
    thread = threading.Thread(target=run_pipeline, args=(sample, results, result_lock))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

# save result to json
notebook_dir = os.getcwd()
dir_path = os.path.join(notebook_dir, "claims_results")
print(dir_path)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
# find the current highest run number
if not os.path.exists(os.path.join(dir_path, "current_run.json")):
    with open(os.path.join(dir_path, "current_run.json"), "w") as f:
        json.dump({model: 0}, f)
with open(os.path.join(dir_path, "current_run.json"), "r") as f:
    runs_dict = json.load(f)
    if runs_dict.get(model) is None:
        run_number = 0
        runs_dict[model] = 0
    else:
        run_number = runs_dict[model]



file_path = os.path.join(dir_path, f"{model.split('/')[-1]}_{run_number}.json")
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False)

runs_dict[model] = run_number + 1
with open(os.path.join(dir_path, "current_run.json"), "w") as f:
    json.dump(runs_dict, f)


Error validating response, retrying: 1 validation error for UrvalsSvar
  Invalid JSON: EOF while parsing a value at line 9497 column 0 [type=json_invalid, input_value='{"språk":"sv","mening":... \t\n  \t\n  \t\n  \t\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid
Response: {"språk":"sv","mening":"Texten framställer den svenska coronastrategin som ett medvetet men problematiskt val att gå för ’flockimmunitet’ – det vill säga att begränsa smittan snarare än att sluta ned samhället eller att spåra varje fall. Enligt utdraget fick Anders Tegnell redan i början av mars starka varningssignaler (t.ex. ett mejl från matematikern Tom Britton om det höga R‑värdet) och det fanns tydliga alternativ: total nedstängning, omfattande smittspårning eller en strategi som bygger på att låta viruset spridas i viss utsträckning för att så småningom uppnå flockimmunitet. Trots dessa varningar och råd från andra framstående läkare (Jan Albert, Johan von S

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI


Client = OpenAI(base_url="http://localhost:4000/v1", api_key="EMPTY")
def get_embeddings(prompts):
    response = Client.embeddings.create(
        model="Qwen/Qwen3-Embedding-8B",
        input=prompts
    )
    return [item.embedding for item in response.data]

print("Embedding claims...")
all_article_claims = []
for id, text_claims in results.items():
    article_claims = []
    embeddings = get_embeddings(text_claims)
    for claim, embedding in zip(text_claims, embeddings):
        article_claims.append({"claim": claim, "id": id, "embedding": embedding})
    all_article_claims.append(article_claims)
print("Done embedding claims")

Embedding claims...


BadRequestError: Error code: 400 - {'error': {'message': '20 validation errors:\n  {\'type\': \'int_type\', \'loc\': (\'body\', \'function-wrap[__log_extra_fields__()]\', \'input\', \'list[int]\', 0), \'msg\': \'Input should be a valid integer\', \'input\': {\'claim\': \'Texten ger intrycket av en medveten, passiv strategi som försöker ligga kvar i en taktil allians gentemot de politiska aktörerna, där Tegnell var navet och där tidiga varningar förbisedes.\', \'id\': \'1324492047340965888\'}}\n  {\'type\': \'int_type\', \'loc\': (\'body\', \'function-wrap[__log_extra_fields__()]\', \'input\', \'list[int]\', 1), \'msg\': \'Input should be a valid integer\', \'input\': {\'claim\': \'Den svenska strategi för COVID‑19 visade tydligt en lax och reaktiv hantering där konsultationer med experter bara resulterade i beslut när situationen hade nått en kritisk fas.\', \'id\': \'1324492047340965888\'}}\n  {\'type\': \'int_type\', \'loc\': (\'body\', \'function-wrap[__log_extra_fields__()]\', \'in

In [ ]:
all_articles_embedding_groups = []
for article_claims in all_article_claims:
    embedding_groups = []
    for claim in article_claims:
        if len(embedding_groups) == 0:
            embedding_groups.append({"representative": {"claim": claim}, "members": [claim]})
        else:
            most_similar_group = max(embedding_groups, key=lambda group: cosine_similarity([embedding], [group["representative"]["embedding"]])[0])
            if most_similar_group is not None and cosine_similarity([embedding], [most_similar_group["representative"]["embedding"]])[0] > 0.65:
                most_similar_group["members"].append(claim)
            else:
                embedding_groups.append({"representative": {"claim": claim}, "members": [claim]})

final_claims = []
for article in all_articles_embedding_groups:
    for article_embedding_group in article:
        representative = article_embedding_group["representative"]["claim"]
        representative["count"] = len(article_embedding_group["members"])
        final_claims.append(representative)


Representative: Det framkommer att Folkhälsomyndigheten betraktades som alltför passiv och att viktiga medicinska röster tvingades insistera på att träffa Anders Tegnell för att påminna om den akuta risken med viruset.
Members: 1
Representative: Det indikerar att den officiella covid‑strategin i Sverige saknade en snabb, proaktiv hållning och därför behövde kompletteras av interna varningar från framstående infektions‑ och smittskyddsexperter.
Members: 7
Representative: Sveriges covid‑19‑strategi var i praktiken ett medvetet val av den tredje möjliga vägen – att låta viruset spridas för att nå herd immunity – en strategi som författaren beskriver som både uppgiven och huvudlös.
Members: 13
Representative: Strategin sägs ha satts i verket trots varningar om höga dödstal och presenterar en ”huvudlös” linje med lågt R‑värde‑klimat.
Members: 1
Representative: Myndigheten var otymplig i att omsätta expertvarningarna i konkreta, tidiga restriktioner.
Members: 1
Representative: Strategin port

In [ ]:
data_result_path = os.path.join(os.getcwd(), "../Data-Analysis-Project-28/data_for_git/extracted_claims.json")
with open(data_result_path, "w", encoding="utf-8") as f:
    json.dump(final_claims, f, ensure_ascii=False)